In [92]:
import esdl
from esdl.esdl_handler import EnergySystemHandler
import plotly.graph_objects as go

In [93]:
sourceList = []
sinkList = []
valueList = []
nodeColourList = []
linkColourList = []
sumList = []
pinkhex = "rgba(215, 198, 225, 0.5)"
pinknode = "#9C75B9"
purplehex = "rgba(205, 208, 236, 0.5)"
purplenode = "#8892D0"
darkbluehex = "rgba(198, 222, 233, 0.5)"
darkbluenode = "#7FAEDF"
lightbluehex = "rgba(200, 236, 223, 0.5)"
lighbluenode = "#7CD6C8"
lightgreennode = "#8EE0A7"
wattageconvertor = 3.6

esh = EnergySystemHandler()
# esh.load_file('../ESDLs-Input/Tholen_Input.esdl')
esh.load_file('../output/2023-05-12T09:17:22.297061/Tholen_Input_A=1_T=1')
energySystem = esh.get_energy_system()
instance = energySystem.instance[0]
topLevelArea = instance.area
AssetList = topLevelArea.asset

FileNotFoundError: [Errno 2] No such file or directory: '../output/2023-05-12T09:17:22.297061/Tholen_Input_A=1_T=1'

In [ ]:
for asset in AssetList:
    if isinstance(asset, esdl.PVInstallation) and asset.state == esdl.AssetStateEnum.OPTIONAL:
        print(asset.power)
        sourceList.append(6)
        sinkList.append(1)
        valueList.append(asset.power * wattageconvertor)
        linkColourList.append(pinkhex)

In [ ]:
for asset in AssetList:
    if isinstance(asset, esdl.AbstractBuilding):
        for a in asset.asset:
            if isinstance(a, esdl.ElectricityDemand):
                for port in a.port:
                    if isinstance(port, esdl.InPort):
                        for profile in port.profile:
                            remainingMJ = 0

                            if a.name == "Gebouwgebonden elektriciteitsgebruik":
                                sourceList.append(3)
                                sinkList.append(4)
                                valueList.append(profile.multiplier)
                                print(a.name, profile.multiplier)
                                linkColourList.append(lightbluehex)

                                remainingMJ += profile.multiplier
                            print("RMJ", remainingMJ)

                            if a.name == "Procesgebonden elektriciteitsgebruik":
                                sourceList.append(3)
                                sinkList.append(5)
                                valueList.append(profile.multiplier)
                                print(profile.multiplier)
                                linkColourList.append(lightbluehex)

                                remainingMJ = remainingMJ + profile.multiplier
                            print("RMJ2", remainingMJ)

                            for a in asset.asset:
                                if isinstance(a, esdl.PVInstallation):
                                    wattGenerated = a.power
                                    MJGenerated = wattGenerated * wattageconvertor
                                    print("MJG", MJGenerated)
                                    sourceList.append(7)
                                    sinkList.append(3)
                                    valueList.append(min(remainingMJ, MJGenerated))
                                    linkColourList.append(pinkhex)
                                    remainingMJ = max(0, remainingMJ - MJGenerated)
                            print(remainingMJ)

                            sourceList.append(2)
                            sinkList.append(3)
                            valueList.append(remainingMJ)
                            linkColourList.append(darkbluehex)

                            sumList.append(remainingMJ)

In [ ]:
sum(sumList)
sourceList.append(0)
sinkList.append(1)
valueList.append(sum(sumList)-valueList[0])
linkColourList.append(pinkhex)

In [ ]:
sum(sumList)
sourceList.append(1)
sinkList.append(2)
valueList.append(sum(sumList))
linkColourList.append(purplehex)

In [ ]:
label = ["Elektricity Supply",
        "Electricity Cable",
        "Electricity Network",
        "Electricity Connector",
        "Gebouwgebonden Eletriciteitgebruik", "Procesgebonden Elektriciteitsgebruik",
        "PV Installation TEACOS",
        "PV Panel Agents"
        ]

nodeColourList = [pinknode,
                  purplenode,
                  darkbluenode,
                  lighbluenode,
                  lightgreennode, lightgreennode,
                  pinknode,
                  pinknode
                  ]

x = [0.0, 0.2, 0.4, 0.6, 0.8, 0.8, 0.0, 0.0]
y = [0.2, 0.5, 0.5, 0.5, 0.2, 0.8, 0.5, 0.8]

link = dict(source = sourceList, target = sinkList, value = valueList, color = linkColourList)
node = dict(label = label, x = x, y = y, pad = 150, thickness = 10, color = nodeColourList)
data = go.Sankey(link = link, node = node)
fig = go.Figure(data)
fig.update_layout(title_text="Sankey Diagram of", font_size=10)
fig.show()